In [9]:
from op_analytics.datasources.chainsrpc.erc20metadata import get_rpc_for_chain

get_rpc_for_chain(10)

'https://mainnet.optimism.io'

In [18]:
import requests

RPC_URL = 'https://mainnet.base.org/'

# Contract details
CONTRACT_ADDRESS = "0xffcbf84650ce02dafe96926b37a0ac5e34932fa5"
CONTRACT_ADDRESS = "0x833589fcd6edb6e08f4c7c32d4f71b54bda02913"
METHOD_ID = "0x18160ddd"  # First 4 bytes of keccak256(function_signature)

# Construct JSON-RPC payload
payload = {
    "jsonrpc": "2.0",
    "method": "eth_call",
    "params": [{
        "to": CONTRACT_ADDRESS,
        "data": METHOD_ID
    }, "latest"],
    "id": 1
}

# Send request
response = requests.post(RPC_URL, json=payload)
result = response.json()
result

{'jsonrpc': '2.0',
 'result': '0x000000000000000000000000000000000000000000000000000ce927c84a2485',
 'id': 1}

In [20]:
int("0x000000000000000000000000000000000000000000000000000ce927c84a2485", base=16)

3634056793826437

3634056793826437

In [11]:
import web3

contract_addres = "0x833589fcd6edb6e08f4c7c32d4f71b54bda02913"

# Create web3 connection
w3 = web3.Web3(web3.Web3.HTTPProvider(rpc))

print(contract_addres)
print(w3.to_checksum_address(contract_addres))

0x833589fcd6edb6e08f4c7c32d4f71b54bda02913
0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913


In [8]:
import web3

contract_addres = "0x833589fcd6edb6e08f4c7c32d4f71b54bda02913"

# Create web3 connection
w3 = web3.Web3(web3.Web3.HTTPProvider(rpc))

# Convert address to checksum format
contract_address = w3.to_checksum_address(contract_addres)

# Implementation slot for ERC1967 proxy
IMPLEMENTATION_SLOT = "0x360894a13ba1a3210667c828492db98dca3e2076cc3735a920a3ca505d382bbc"

# Get implementation address from storage
implementation_bytes = w3.eth.get_storage_at(contract_address, IMPLEMENTATION_SLOT)
print(implementation_bytes)
implementation_address = "0x" + implementation_bytes.hex()[-40:]

print(f"Implementation address: {implementation_address}")


b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Implementation address: 0x0000000000000000000000000000000000000000


In [1]:
from op_analytics.datasources.chainsrpc.erc20metadata import execute_pull


results = execute_pull()

2025-02-13 15:55:47 [debug    ] connecting to OPLABS Clickhouse client... filename=client.py lineno=56 process=44945
2025-02-13 15:55:47 [info     ] loaded vault from .env file    filename=vault.py lineno=32 process=44945
2025-02-13 15:55:47 [debug    ] loaded vault: 18 items         filename=vault.py lineno=79 process=44945
2025-02-13 15:55:47 [debug    ] initialized OPLABS Clickhouse client. filename=client.py lineno=61 process=44945
2025-02-13 15:55:47 [debug    ] Found vault variable GOOGLE_SERVICE_ACCOUNT (has JSON key) counter=001/010 eta=None filename=gcpauth.py lineno=18 process=44945
2025-02-13 15:55:47 [debug    ] Found vault variable GOOGLE_SERVICE_ACCOUNT (has JSON key) counter=002/010 eta=None filename=gcpauth.py lineno=18 process=44945
2025-02-13 15:55:47 [debug    ] Found vault variable GOOGLE_SERVICE_ACCOUNT (has JSON key) counter=003/010 eta=None filename=gcpauth.py lineno=18 process=44945
2025-02-13 15:55:47 [debug    ] Found vault variable GOOGLE_SERVICE_ACCOUNT (has

In [2]:
results

{TokenContract(chain='worldchain', chain_id=480, contract_address='0x2cfc85d8e48f8eab294be644d9e25c3030863003'): {},
 TokenContract(chain='base', chain_id=8453, contract_address='0x6bbdcaae8533b0e8e9c11e6ddea43b8d88e263f6'): {},
 TokenContract(chain='base', chain_id=8453, contract_address='0x940181a94a35a4569e4529a3cdfb74e38fd98631'): {},
 TokenContract(chain='base', chain_id=8453, contract_address='0x773e90176fd8c8335b8692a8c3c6e777d640a049'): {},
 TokenContract(chain='worldchain', chain_id=480, contract_address='0xcd1e32b86953d79a6ac58e813d2ea7a1790cab63'): {},
 TokenContract(chain='base', chain_id=8453, contract_address='0xae56028079fb8e3fc788b25af7e4e343f7878e4c'): {},
 TokenContract(chain='base', chain_id=8453, contract_address='0x4200000000000000000000000000000000000006'): {'chain': 'base',
  'chain_id': 8453,
  'contract_address': '0x4200000000000000000000000000000000000006',
  'decimals': 18,
  'symbol': 'WETH',
  'name': 'Wrapped Ether',
  'total_supply': 240514515674559726527

In [3]:
import pandas as pd
import sys
sys.path.append("../helper_functions")
from web3 import Web3
from datetime import datetime, timezone
sys.path.pop()
import re

In [4]:
def clean_string(raw_str: str) -> str:
    """
    Remove all control characters (ASCII 0–31, plus 127),
    then strip leading/trailing whitespace.
    """
    return re.sub(r'[\x00-\x1F\x7F]+', '', raw_str).strip()


def get_token_metadata(token_contract_address, rpc, method_name_list):
    """
    Aggregates multiple contract calls into one Python function call.
    (Still multiple calls to your Ethereum node, but simpler to manage in code.)
    """
    w3_conn = Web3(Web3.HTTPProvider(rpc))
    token_contract_address = token_contract_address.lower() 
    
    block_timestamp = w3_conn.eth.get_block('latest').timestamp
    block_number = w3_conn.eth.get_block('latest').number
    # Convert the UNIX timestamp to a human-readable format
    block_datetime = datetime.fromtimestamp(block_timestamp, tz=timezone.utc)
    block_time = block_datetime.strftime('%Y-%m-%d %H:%M:%S')

    method_ids = [Web3.keccak(text=f"{method_name}()")[:4].hex() for method_name in method_name_list]
    
    # create a loop to get the response for each method
    response_map = {}

    for method_id, method_name in zip(method_ids, method_name_list):
        response = w3_conn.eth.call({
            'to': token_contract_address,
            'data': method_id
        })
        response_map[method_name] = response


    # return the response for each method
    
    decimals = Web3.to_int(hexstr=response_map["decimals"].hex())
    symbol = clean_string(Web3.to_text(hexstr=response_map["symbol"].hex()))
    name = clean_string(Web3.to_text(hexstr=response_map["name"].hex()))
    total_supply = Web3.to_int(hexstr=response_map["totalSupply"].hex())
    
    return {
        "block_time": block_time,
        "block_number": block_number,
        "block_timestamp": block_timestamp,
        "contract_address": token_contract_address,
        "decimals": decimals,
        "symbol": symbol,
        "name": name,
        "total_supply": total_supply
    }

In [5]:
chains_rpcs = pd.read_csv('../../op_chains_tracking/outputs/chain_metadata.csv', na_filter=False)
chains_rpcs = chains_rpcs[~(chains_rpcs['rpc_url'] == '')]

rpc =chains_rpcs[chains_rpcs['chain_name'] == 'op']["rpc_url"]
RPC = rpc.iloc[0] 

CONTRACT_ADDRESS = "0x4200000000000000000000000000000000000006"
METHOD_NAME = ["decimals", "symbol", "name", "totalSupply"]


In [14]:
response = get_token_metadata(CONTRACT_ADDRESS, RPC, METHOD_NAME)
response

{'block_time': '2025-02-13 21:06:07',
 'block_number': 131940995,
 'block_timestamp': 1739480767,
 'contract_address': '0x4200000000000000000000000000000000000006',
 'decimals': 18,
 'symbol': 'WETH',
 'name': 'Wrapped Ether',
 'total_supply': 38885309302187337741110}

In [ ]:
address2 = Web3.to_checksum_address('0xffcbF84650cE02DaFE96926B37a0ac5E34932fa5')
balance=contract.functions.balanceOf(address2).call()

In [16]:
contract = "0xffcbF84650cE02DaFE96926B37a0ac5E34932fa5"

response = get_token_metadata(contract, RPC, METHOD_NAME)
response

InvalidAddress: ('web3.py only accepts checksum addresses. The software that gave you this non-checksum address should be considered unsafe, please file it as a bug on their platform. Try using an ENS name instead. Or, if you must accept lower safety, use Web3.to_checksum_address(lower_case_address).', '0xffcbf84650ce02dafe96926b37a0ac5e34932fa5')

In [13]:
checksum_addr = w3.to_checksum_address("0xeb585163debb1e637c6d617de3bef99347cd75c8")
print(checksum_addr)

response = get_token_metadata(checksum_addr, "https://mainnet.optimism.io", METHOD_NAME)

0xeB585163DEbB1E637c6D617de3bEF99347cd75c8


InvalidAddress: ('web3.py only accepts checksum addresses. The software that gave you this non-checksum address should be considered unsafe, please file it as a bug on their platform. Try using an ENS name instead. Or, if you must accept lower safety, use Web3.to_checksum_address(lower_case_address).', '0xeb585163debb1e637c6d617de3bef99347cd75c8')

In [8]:
response = get_token_metadata("0x2cfc85d8e48f8eab294be644d9e25c3030863003", "https://worldchain-mainnet.g.alchemy.com/public", METHOD_NAME)

InvalidAddress: ('web3.py only accepts checksum addresses. The software that gave you this non-checksum address should be considered unsafe, please file it as a bug on their platform. Try using an ENS name instead. Or, if you must accept lower safety, use Web3.to_checksum_address(lower_case_address).', '0x2cfc85d8e48f8eab294be644d9e25c3030863003')